In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

def load_pdf_data(pdf_path):
    """
    this function loads text data from pdf file
    """
    loader = PyPDFLoader(file_path=pdf_path)
    documents = loader.load()
    return documents

In [ ]:
general = load_pdf_data(pdf_path = '/Users/sayo/personal_projects/Usafe_bot/data/general.pdf')

In [ ]:

print(f"number of loaded pages: {len(general)}")

In [ ]:
print("________________")
print(general[0].page_content)

### Split Document into Chunks

>- not possible to feed the whole content into the LLM at once because of finite context window
>- even models with large window sizes may struggle to find information in very long inputs and perform very badly
>- chunk the document into pieces: helps retrieve only the relevant information from the corpus

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=800, chunk_overlap=80):
    """
    this function splits documents into chunks of given size and overlap
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents=documents)
    return chunks

In [ ]:
general_chunks = split_documents(general)
print(f"number of chunks: {len(general_chunks)}")

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

def create_embedding_vector_db(chunks, db_name):
    """
    this function uses the open-source embedding model HuggingFaceEmbeddings 
    to create embeddings and store those in a vector database called FAISS, 
    which allows for efficient similarity search
    """
    # instantiate embedding model
    embedding = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2'
    )
    # create the vector store 
    vectorstore = FAISS.from_documents(
        documents=chunks,
        embedding=embedding
    )
    # save vector database locally
    vectorstore.save_local(f"./vector_databases/vector_db_{db_name}")

i can combine all of them above. on top of description add the definitions (one pdf each)

In [ ]:
create_embedding_vector_db(chunks=general_chunks, db_name='general')

## Retrieve from Vector Database

In [ ]:
def retrieve_from_vector_db(vector_db_path):
    """
    this function spits out a retriever object from a local vector database
    """
    # instantiate embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2'
    )
    react_vectorstore = FAISS.load_local(
        folder_path=vector_db_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True
    )
    retriever = react_vectorstore.as_retriever()
    return retriever

In [ ]:
general_retriever = retrieve_from_vector_db(vector_db_path='./vector_databases/vector_db_general')

In [ ]:
type(general_retriever)

#### Load the prompt

In [ ]:
with open('/Users/sayo/personal_projects/Usafe_bot/data/usafe_prompt.txt', 'r') as file:
    user_prompt = file.read()

print(user_prompt)

## Generation

[`create_stuff_documents_chain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html#langchain.chains.combine_documents.stuff.create_stuff_documents_chain)

- takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM
- passes ALL documents, so you should make sure it fits within the context window of the LLM being used

In [ ]:
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain

**chain passing user inquiry to retriever object**

[`create_retrieval_chain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html#langchain.chains.retrieval.create_retrieval_chain)

- takes in a user inquiry, which is then passed to the retriever to fetch relevant documents
- those documents (and original inputs) are then passed to an LLM to generate a response

In [ ]:
from langchain.chains.retrieval import create_retrieval_chain

connect chains

In [ ]:
def connect_chains(retriever):
    """
    this function connects stuff_documents_chain with retrieval_chain
    """
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.02,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

output generation

In [ ]:
react_retrieval_chain = connect_chains(general_retriever)

In [ ]:
def print_output(
    inquiry,
    retrieval_chain=react_retrieval_chain
):
    result = retrieval_chain.invoke({"input": inquiry})
    print(result['answer'].strip("\n"))

Rakib: replace answer. Make print(result). It will give all the info, it will tell which chunk. It will give the info of which pdf is coming from. 

3 pdfs for each hate crime. Be able to find the correct hate type. first embedding then rag. 

In [ ]:
print_output("what is hate crime")

In [ ]:
print_output("quais sao os crimes de odio")

In [ ]:
print_output("What are the laws in germany to regards hate crime agaisnt trans people")

In [ ]:
print_output("what is the advice you can give to someone who is a victim of hate crime")

In [ ]:
print_output("i was victim of a hate crime, what should i do?")

In [ ]:
print_output("How do I report a hate crime in Germany?")

In [ ]:
print_output("give me the list of all resources")

In [ ]:
print_output("what is the history of hate crime")

In [ ]:
print_output('what are the consequences of hate crime')

In [ ]:
print_output("i was assaulted because i'm black")